## 基于用户的协同过滤 UCF

协同过滤类算法有三个关键问题，分别是：  
1. 如何计算相似度，用户与用户、物品与物品间的
2. 如何选择最相近的K个用户或者物品
3. 如何基于相近用户或物品预测当前评分

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from tqdm import tqdm

In [2]:
# 每个用户的评分的平均值
u_r_averages = {}
# 每个用户评分过的电影ID
u_r_items = {}
#每个电影评分过的用户id列表
item_rating_lists = {}
all_data = pd.read_csv("datasets/ml-100k/u.data",sep='\t',names=['uid','iid','rating','time'])
train_data = pd.read_csv("datasets/ml-100k/u1.base",sep='\t',names=['uid','iid','rating','time'])
test_data = pd.read_csv("datasets/ml-100k/u1.test",sep='\t',names=['uid','iid','rating','time'])

In [3]:
for i,group in train_data.groupby('uid'):
    u_r_averages[i] = group['rating'].mean()
    u_r_items[i] = set(group['iid'])
# 建立倒排表：电影->用户
for i,group in train_data.groupby('iid'):
    item_rating_lists[i] = set(group['uid'])

In [4]:
ratings = {}
for i,group in train_data.groupby('uid'):
    rating_dict = dict(zip(group['iid'], group['rating']))
    ratings[i] = rating_dict

In [5]:
rating_arr = [[0]*1682 for i in range(943)]
for i,group in train_data.groupby('uid'):
    arr = zip(group['iid'], group['rating'])
    for pair in arr:
        rating_arr[i-1][pair[0]-1] = pair[1]

In [6]:
test_iid_set = set(test_data['iid'])
train_iid_set = set(train_data['iid'])

In [7]:
new_item_set = test_iid_set - train_iid_set

In [10]:
import numpy as np
from scipy.stats import pearsonr
 
# 示例数据
x = np.array(rating_arr[0])
y = np.array(rating_arr[1])
 
# 计算皮尔逊相关系数
coef, p_value = pearsonr(x,y)
 
print(f"皮尔逊相关系数: {coef}, 显著性水平p-value: {p_value}")

皮尔逊相关系数: 0.05987606750043184, 显著性水平p-value: 0.014048260157516295


In [30]:
similar_pcc = np.corrcoef(rating_arr)

In [51]:
resu[0]

array([ 1.00000000e+00,  5.98760675e-02,  2.16428733e-02, -2.78994605e-03,
        1.45654752e-01,  2.40147977e-01,  1.20057048e-01,  6.68248385e-02,
        4.04940402e-02,  1.47571741e-01,  1.02584142e-01,  1.45454804e-01,
        1.56611364e-01,  1.18692930e-01,  1.01845715e-01,  2.04649189e-01,
        9.60659243e-02,  2.33142909e-01,  3.22623806e-02,  8.85236877e-02,
        8.04039608e-02,  1.37910360e-01,  2.05116118e-01,  1.62309893e-01,
        1.18155627e-01,  7.45675441e-02,  1.09885778e-01,  3.56275900e-02,
        5.16897366e-02,  6.46396554e-02,  7.74899443e-02,  1.34488764e-01,
       -1.79116817e-03, -1.96795174e-02, -2.13856412e-02, -1.04419329e-02,
        9.95773528e-02, -1.18684849e-03,  4.04449199e-02,  1.40628540e-02,
        2.03351270e-01,  1.58384521e-01,  1.40568968e-01,  2.06084774e-01,
        1.10308442e-01,  5.29760768e-02,  4.95985445e-03,  2.24311362e-02,
        8.70370953e-02,  3.61133963e-02,  2.12428945e-01,  1.32140415e-01,
        1.22049788e-01,  

In [63]:
similar_arr[0]

array([ 1.00000000e+00,  5.98760675e-02,  2.16428733e-02, -2.78994605e-03,
        1.45654752e-01,  2.40147977e-01,  1.20057048e-01,  6.68248385e-02,
        4.04940402e-02,  1.47571741e-01,  1.02584142e-01,  1.45454804e-01,
        1.56611364e-01,  1.18692930e-01,  1.01845715e-01,  2.04649189e-01,
        9.60659243e-02,  2.33142909e-01,  3.22623806e-02,  8.85236877e-02,
        8.04039608e-02,  1.37910360e-01,  2.05116118e-01,  1.62309893e-01,
        1.18155627e-01,  7.45675441e-02,  1.09885778e-01,  3.56275900e-02,
        5.16897366e-02,  6.46396554e-02,  7.74899443e-02,  1.34488764e-01,
       -1.79116817e-03, -1.96795174e-02, -2.13856412e-02, -1.04419329e-02,
        9.95773528e-02, -1.18684849e-03,  4.04449199e-02,  1.40628540e-02,
        2.03351270e-01,  1.58384521e-01,  1.40568968e-01,  2.06084774e-01,
        1.10308442e-01,  5.29760768e-02,  4.95985445e-03,  2.24311362e-02,
        8.70370953e-02,  3.61133963e-02,  2.12428945e-01,  1.32140415e-01,
        1.22049788e-01,  

### PCC计算，皮尔逊相关系数
改进思路：  
1：对于热门物品，若两个用户评分相近，则其权值应该较低，若两个用户对一个冷门物品评分相近，则权值较高  
2：若两个用户co-rated item较少，则整个相似度的置信度应该较低

In [53]:
'''
ratings列表：是评分表， ratings[i][j]表示用户ID=i+1对电影ID=j+1的评分
u_r_items字典：每个用户评分过的电影ID
u_r_averages字典：每个用户的评分的平均值
''' 
def pcc(ratings, u_r_items, u_r_averages, user_num):
    user_set = u_r_averages.keys()
    pcc_arr = [[1]*user_num for _ in range(user_num)]
    for uid in tqdm(user_set):
        # uid 和 pair表示此时计算这两个用户的PCC
        for pair in user_set:
            # 最终形成一个长宽为用户数的矩阵
            if uid == pair:
                continue
            if pcc_arr[pair-1][uid-1] != 1:
                pcc_arr[uid-1][pair-1] = pcc_arr[pair-1][uid-1]
                continue
            Numerator = 0
            denominator_one = 0
            denominator_two = 0
            common_rating_items = u_r_items.get(uid) & u_r_items.get(pair)
            for iid in common_rating_items:
                uid_rating = ratings.get(uid).get(iid)
                pair_rating = ratings.get(pair).get(iid)
                bias_uid = uid_rating - u_r_averages.get(uid)
                bias_pair = pair_rating - u_r_averages.get(pair)
                # 分子
                Numerator += (bias_uid * bias_pair)*1.0
                # 分母
                denominator_one += pow(bias_uid,2)*1.0
                denominator_two += pow(bias_pair,2)*1.0
            denominator_one = math.sqrt(denominator_one)
            denominator_two = math.sqrt(denominator_two)
            if denominator_one == 0 or denominator_two == 0:
                continue
#                 看到其他博客的方法，不知道为什么 https://blog.csdn.net/recsysml/article/details/12287587
#             if denominator_one == 0:
#                 denominator_one = 0.001
#             if denominator_two == 0:
#                 denominator_two = 0.001
            pcc_value = Numerator / (denominator_one * denominator_two)
            # 这里可能有一些值略微大于1，例如1.000000002
            if abs(pcc_value) > 1:
                pcc_value = int(pcc_value)
            pcc_arr[uid-1][pair-1] = pcc_value
    return pcc_arr

## 邻居选择
改进思路：假设K为20，如果前18个相似度较高，但最后2个较低，可以舍去最后2个，即设置一个相似度阈值，与K一起作用

In [54]:
'''
pcc_arr 列表：pcc_arr[i][j] 表示用户ID=i+1与用户ID=j+1 的相关系数，只记录了上半三角，必须从小索引先开始
item_rating_lists字典：item_rating_lists[i]表示对电影ID=i评分过的用户ID集合
'''
def ucf_neigh_select(pcc_arr,item_rating_lists, k, uid, iid):
    recList = {}
    # 对电影iid评过分的用户ID，从1开始的
    item_rated_users = item_rating_lists.get(iid)
    # 和用户uid的PCC不等于0的用户ID集合，从1开始的
    pcc_non_zero = set([index+1 for index, value in enumerate(pcc_arr[uid-1]) if value != 0.0])
    # 去除自身
    pcc_non_zero.remove(uid)
    if item_rated_users==None:
        item_rated_users = set()
    # 正常的用户ID，从1开始。这里已经排除了自身
    common_users = item_rated_users & pcc_non_zero
    if len(common_users)==0:
        return dict()
    # 选择这里面pcc最高的K个用户
    for i in common_users:
        # 因为PCC矩阵只保留了上三角，所以要从小的下标开始索引
        small_id = min(uid,i)
        big_id = max(uid,i)
        recList[i] = pcc_arr[small_id-1][big_id-1]
    topK = dict(sorted(recList.items(), key=lambda x: x[1],reverse = True))
    if len(topK) > k:
        return dict(list(topK.items())[:k])
    else:
        return topK

## 验证函数

In [55]:
def testUCF(path, pcc_arr, item_rating_lists, k):
    predicts = []
    targets = []
    with open(path, "r") as file:
        lines = file.readlines()
        for i in range(len(lines)):
            predict_u_r = 0
            user_bias = 0
            sum_pcc = 0
            uid = int(lines[i].split()[0])
            iid = int(lines[i].split()[1])
            # 冷启动处理，测试集中的物品在训练集没有出现过，则跳过该物品的评分预测
            if iid in new_item_set:
                continue
            target_rating = int(lines[i].split()[2])
            user_rating_average = u_r_averages.get(uid)
            
            top_k_dict = ucf_neigh_select(pcc_arr, item_rating_lists, k, uid, iid)
            if len(top_k_dict) == 0:
                predict_u_r = user_rating_average
            else:
                for userId,pcc in top_k_dict.items():
                    user_bias += ((ratings.get(userId).get(iid) - u_r_averages.get(userId)) * pcc)
                    sum_pcc += abs(pcc)
                user_bias = user_bias / (sum_pcc)
                predict_u_r = user_rating_average + user_bias
            # 后处理过程
            if predict_u_r > 5:
                predict_u_r = 5
            if predict_u_r < 1:
                predict_u_r = 1
            
            predicts.append(predict_u_r)
            targets.append(target_rating)
    return targets,predicts

In [61]:
# similar_arr = pcc(ratings, u_r_items, u_r_averages,943)
similar_arr = np.corrcoef(rating_arr)
a,b = testUCF("datasets/ml-100k/u1.test", similar_arr, item_rating_lists, 50)

In [62]:
from sklearn.metrics import mean_squared_error,mean_absolute_error
print('MSE={}'.format(math.sqrt(mean_squared_error(a,b))))
print('MAE={}'.format(mean_absolute_error(a,b)))

MSE=0.9561858486943877
MAE=0.7492020495442102


### ICF

## ACS （adjusted cosine similarity）经过调整的余弦相似度

In [67]:
'''
ratings列表：是评分表， ratings[i][j]表示用户ID=i+1对电影ID=j+1的评分
item_rating_lists：每个电影评分过的用户ID
u_r_averages字典：每个用户的评分的平均值
''' 
def acs(ratings, item_rating_lists, u_r_averages, item_num):
    iid_set = item_rating_lists.keys()
    acs_arr = [[1]*item_num for _ in range(item_num)]
    for k in tqdm(iid_set):
        for j in iid_set:
            if k == j:
                continue
            # 如果另一对已经做过，则不用去做了    
            if acs_arr[j-1][k-1] != 1:
                acs_arr[k-1][j-1] = acs_arr[j-1][k-1]
                continue
            numerator = 0
            denominator_one = 0
            denominator_two = 0
            common_rating_users = item_rating_lists.get(k) & item_rating_lists.get(j)
            for uid in common_rating_users:
                bias_k = ratings.get(uid).get(k) - u_r_averages.get(uid)
                bias_j = ratings.get(uid).get(j) - u_r_averages.get(uid)
                numerator += round(bias_k * bias_j,4)
                denominator_one += round(bias_k * bias_k,4)
                denominator_two += round(bias_j * bias_j,4)
            denominator_one = math.sqrt(denominator_one)
            denominator_two = math.sqrt(denominator_two)
            if denominator_one == 0 or denominator_two == 0:
                    continue
            acs_value = numerator / round(denominator_one * denominator_two,4)
            # acs 值在[-1，1]区间            
            if abs(acs_value) > 1:
                acs_value = int(acs_value)
            acs_arr[k-1][j-1] = acs_value
    return acs_arr

In [8]:
'''
acs_arr 列表：acs_arr[k][j] 表示物品k与物品j的相关系数，只记录了上半三角，必须从小索引先开始k<j
item_rating_lists字典：item_rating_lists[i]表示对电影ID=i评分过的用户ID集合
'''
def icf_neigh_select(acs_arr,u_r_items, k, uid, iid):
    recList = {}
    # 用户评过分的物品
    user_rated_items = u_r_items.get(uid)
    # 和物品iid的ACS不等于0的物品ID集合，从1开始的
    acs_non_zero = set([index+1 for index, value in enumerate(acs_arr[iid-1]) if value != 0])
    if user_rated_items == None:
        user_rated_items = set()
    common_items = user_rated_items & acs_non_zero
    if len(common_items)==0:
        return dict()
    for i in common_items:
        # 因为相似度矩阵只保留了上三角，所以要从小的下标开始索引
        small_id = min(iid,i)
        big_id = max(iid,i)
        recList[i] = acs_arr[small_id-1][big_id-1]
    topK = dict(sorted(recList.items(), key=lambda x: x[1],reverse = True))
    if len(topK) > k:
        return dict(list(topK.items())[:k])
    else:
        return topK

In [9]:
def testICF(path, acs_arr, u_r_items, k):
    predicts = []
    targets = []
    with open(path, "r") as file:
        lines = file.readlines()
        for i in range(len(lines)):
            predict_u_r = 0
            user_bias = 0
            sum_acs = 0
            uid = int(lines[i].split()[0])
            iid = int(lines[i].split()[1])
#             if iid in setC:
#                 continue
            target_rating = int(lines[i].split()[2])
            user_rating_average = u_r_averages.get(uid)
            
            top_k_dict = icf_neigh_select(acs_arr, u_r_items, k, uid, iid)
            if len(top_k_dict) == 0:
                predict_u_r = user_rating_average
            else:
                for item_id,acs in top_k_dict.items():
                    user_bias += ratings.get(uid).get(item_id) * acs
                    sum_acs += acs
                # uid=130 iid=1273 top_k_dict = {1278: 1.0, 1279: -1.0}
                if sum_acs == 0:
                    print(uid,iid,top_k_dict)
                    continue
                user_bias = user_bias / (sum_acs)
                predict_u_r = user_bias
            # 后处理过程
            if predict_u_r > 5:
                predict_u_r = 5
            if predict_u_r < 1:
                predict_u_r = 1
            
            predicts.append(predict_u_r)
            targets.append(target_rating)
    return targets,predicts

In [68]:
similar_arr = acs(ratings, item_rating_lists, u_r_averages, 1682)
# a,b = testICF("datasets/ml-100k/u1.test", similar_arr, u_r_items, 50)

100%|██████████████████████████████████████████████████████████████████████████████| 1650/1650 [00:19<00:00, 83.15it/s]


In [11]:
from sklearn.metrics.pairwise import cosine_similarity

返回数组的第i行第j列表示a[i]与a[j]的余弦相似度

In [41]:
# a=[[1,3,2],[2,2,1]]
transposed_matrix = list(map(list, zip(*rating_arr)))
result = cosine_similarity(transposed_matrix)

In [70]:
similar_arr[0][0]

1

In [62]:
import numpy as np
vec1 = np.array(transposed_matrix[0])
vec2 = np.array(transposed_matrix[1])

cos_sim = vec1.dot(vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
print(cos_sim)

0.3576463770676817


In [59]:
a,b = testICF("datasets/ml-100k/u1.test", result, u_r_items, 50)

In [60]:
from sklearn.metrics import mean_squared_error,mean_absolute_error
print('MSE={}'.format(math.sqrt(mean_squared_error(a,b))))
print('MAE={}'.format(mean_absolute_error(a,b)))

MSE=1.0129505102076473
MAE=0.7953095140729466
